In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


PATH='/home/wk/myProjects/data/Enron/oper/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

bs=24; bptt=100
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

learner.load_encoder('adam3_10_enc')

In [2]:
#PATH=Path('/home/wk/myProjects/data/Enron/tag/')

In [3]:
#[str(i) for i in PATH.iterdir()]
#PATH/'here.txt'.open()

In [46]:
### Test
ss=r"""
Message-ID: <25459858.1075845925938.JavaMail.evans@thyme>
Date: Mon, 24 Jul 2000 11:03:00 -0700 (PDT)
From: kay.mann@enron.com
To: sheila.tweed@enron.com
Subject: Re: VEPCO/FYI
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Kay Mann
X-To: Sheila Tweed
X-cc: 
X-bcc: 
X-Folder: \Kay_Mann_June2001_3\Notes Folders\Sent
X-Origin: MANN-K
X-FileName: kmann.nsf

To tell you the truth, this stuff is difficult for me to retain, so I could 
be completely off base. I would like to have Herman walk through it again.




From: Sheila Tweed@ECT on 07/24/2000 05:30 PM
To: Kay Mann/Corp/Enron@ENRON
cc:  

Subject: Re: VEPCO/FYI  

It makes one wonder whether one can be in any position in these deals and not 
be in a lease.  Maybe we should ask Herman to describe one.  I was under the 
assumption that if the PPA is not plant specific, there is less of a concern 
on who the EPC contractor is.  I also thought the O&M affiliate was fatal 
even if we sold down the equity.  Have I missed the question again?


"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

'message-id: <25459858.1075845925938.javamail.evans@thyme> date: mon, 24 jul 2000 11:03:00 -0700 (pdt) from: kay.mann@enron.com to: sheila.tweed@enron.com subject: re: vepco/fyi mime-version: 1.0 content-type: text/plain; charset=us-ascii content-transfer-encoding: 7bit x-from: kay mann x-to: sheila tweed x-cc: x-bcc: x-folder: \\kay_mann_june2001_3\\notes folders\\sent x-origin: mann-k x-filename: kmann.nsf to tell you the truth, this stuff is difficult for me to retain, so i could be completely off base. i would like to have herman walk through it again. from: sheila tweed@ect on 07/24/2000 05:30 pm to: kay mann/corp/enron@enron cc: subject: re: vepco/fyi it makes one wonder whether one can be in any position in these deals and not be in a lease. maybe we should ask herman to describe one. i was under the assumption that if the ppa is not plant specific, there is less of a concern on who the epc contractor is. i also thought the o&m affiliate was fatal even if we sold down the equity

In [47]:
#https://askubuntu.com/questions/607118/cuda-not-working-after-returning-laptop-from-sleep#750939
len(s[0])

157

In [48]:
#sudo rmmod nvidia_uvm
#sudo modprobe nvidia_uvm

In [49]:
#t.long().cpu()

In [50]:
ss = copy.deepcopy(s)
t=TEXT.numericalize(s)
#t = t.long().cpu()

m=learner.model

m[0].bs=1
m.eval()
m.reset()

#m.cpu()
#m.cuda()
res,*_ = m(t)
for i in range(len(s[0])):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]                      
    if (predict[0:2] == '@@'):
        print("pos=" + str(i) + "; " + predict + " <- " + s[0][i] , end=' \n')                    
        ss[0][i]=predict
        n.data[0] = TEXT.vocab.stoi[predict]
    else:
        n.data[0] = TEXT.vocab.stoi[s[0][i]]
    res,*_ = m(n[0].unsqueeze(0))
print('...')
print(len(t))

pos=1; @@mes_id@@ <- <25459858.1075845925938.javamail.evans@thyme> 
pos=4; @@othr_dt@@ <- 24 
pos=11; @@sndr_email@@ <- kay.mann@enron.com 
pos=13; @@recr_email1@@ <- sheila.tweed@enron.com 
pos=25; @@sndr_fn@@ <- kay 
pos=26; @@sndr_ln@@ <- mann 
pos=28; @@rcvr_1fn@@ <- sheila 
pos=29; @@rcvr_1ln@@ <- tweed 
pos=34; @@sndr_ln@@ <- folders\sent 
pos=36; @@sndr_ln@@ <- mann-k 
pos=70; @@rcvr_1fn@@ <- sheila 
pos=71; @@rcvr_1ln@@ <- tweed@ect 
pos=72; @@othr_dt@@ <- on 
pos=77; @@sndr_fn@@ <- kay 
pos=78; @@sndr_ln@@ <- mann/corp/enron@enron 
...
157


In [9]:
TEXT.vocab.itos[n.data[0]]

'ben'

In [10]:
n[0].unsqueeze(0)

Variable containing:
 558
[torch.cuda.LongTensor of size 1x1 (GPU 0)]

In [11]:
[TEXT.vocab.itos[e] for e in n.data]

['ben', '<eos>']

In [12]:
TEXT.vocab.stoi['and']

5

In [13]:
n[0]

Variable containing:
 558
[torch.cuda.LongTensor of size 1 (GPU 0)]